# Optimization and MCMC

Gareth Funning, University of California, Riverside

Here are examples of using optimization methods (e.g. the Powell algorithm, as included in the SciPy library) and a Markov Chain Monte Carlo $-$ MCMC $-$ method (e.g. the Metropolis algorithm) to solve nonlinear inverse problems. This is pretty fancy stuff $-$ but you should take care that the results you get are sensible and representative!

## 0. Dependencies

Again, make sure you're in the correct conda environment, or this won't work (and you'll have to quit out of this and activate it, and then restart Jupyter...)

In [ ]:
# import your dependencies
import numpy as np
from scipy.optimize import minimize, Bounds
import pygmt

## 1. Some functions

We will reintroduce our 1D slip model here, alongside a couple of other functions that make use of it. One is a penalty function, that computes the total squared misfit. The other is the log-likelihood (or 'log-probability') function, which, it turns out, is just the total squared misfit, $RSS \times -2$.

Note that we put all of the model parameters in a single vector (a numpy array) $-$ this is because the different approaches we will use below expect that, and expect it to be the first input to the penalty and log-probability functions.

We will use these in the exercises below. Optimization methods will try to minimize the penalty function, whereas MCMC methods will try to maximize the log-probability function.

In [ ]:
# our 1d slip model as before
def slip1d(s, D, v_shift, x):
    v=(s/np.pi)*np.arctan(x/D)+v_shift
    return v

# the total squared penalty function
# 'state' here is an array of the model parameters - s, D and v_shift
def penalty(state, x, data):
    s=state[0]
    D=state[1]
    v_shift=state[2]
    v=slip1d(s, D, v_shift, x)
    p=np.sum((data-v)**2)
    return p
    
# the log probability function (-2 * the total squared misfit)
def logprobability(state, x, data):
    p=penalty(state, x, data)
    logp=-2*p
    return logp

## 2. Data and model setup

Here, we will load in the data we used before, and set the upper and lower parameter bounds for the modeling we want to do. 

In [ ]:
# load in the data!

# i took the liberty of just outputting the arrays we used last time
# (minus the obvious outlier)

x_gps=np.array((-20.12591147,  141.0701294 ,   78.25686249,  -55.40876592,
       -103.8866626 ,    9.42240444,   38.98726886,
         14.22365707,  -79.80173625,  -64.04128843,    2.88787778,
        -81.4305956 ,  -30.8734159 ,    7.18245415,  -69.81071468,
         21.67544474,   -2.26853432, -106.16939644,  -15.30398516,
          2.02005263, -106.48341489))

v_gps=np.array((-37.35777473, -16.7157987 , -16.62951891, -42.66468414,
       -47.93899115, -25.60814096, -17.06184387,
       -23.93251853, -48.90754722, -46.93080769, -30.11640353,
       -46.82690154, -41.18189799, -26.69292887, -46.48615705,
       -20.49406783, -33.3267652 , -47.59029564, -37.38865282,
       -32.67695259, -47.6475494))

v_gps_err=np.array((1.48695262, 2.52997939, 0.5908995 , 0.62248426, 1.88520919,
       1.34222111,  0.41498951, 0.44405611, 1.34222111,
       1.68551465, 0.4       , 1.3       , 1.3       , 0.73887019,
       1.4       , 0.66079412, 1.3       , 1.88520919, 0.44895155,
       1.24229384, 1.2))

# let's define the parameter ranges we want to optimize over
smin=25
smax=45
Dmin=10
Dmax=25
v_shiftmin=-38
v_shiftmax=-28

# make arrays of our upper and lower parameter bounds
pars_lb=np.array((smin, Dmin, v_shiftmin))
pars_ub=np.array((smax, Dmax, v_shiftmax))
npars = len(pars_lb)  # number of parameters

## 3. Simple optimization with SciPy and the Powell algorithm

The Powell algorithm is a downhill, direction-set algorithm which is probably overkill for this specific example, but it is pretty straightforward to use it. (The complicated thing, really, is to set up the model in the first place. Once you have a penalty function and bounds established, it's pretty much plug and play to run it.) The `scipy.optimize.minimize` function can be bounded, which is helpful to stop it going wildly off-base. All it requires is the penalty function, the initial state of the model (which we choose at random), the additional inputs (arguments, or `args`) to the penalty function, and the bounds...

In [ ]:
# make a proper bounds object
pars_bounds = Bounds(pars_lb, pars_ub)

# choose a random initial model state
init=pars_lb+(pars_ub-pars_lb)*np.random.rand(npars)

# and run the Powell algorithm minimizer! 
results = minimize(penalty, init, args=(x_gps, v_gps),  method='Powell', 
                   bounds=pars_bounds)

In [ ]:
# and output the results
results

Note that the results are provided in a structure. Some fields of note:

- `results.x` is the best-fitting parameter values
- `results.fun` is the best penalty
- `results.nfev` is the number of times the penalty function was evaluated

Check your results by plotting the fit of the model to your data.

In [ ]:
# this cell is for evaluating and plotting your best-fitting model




What happens if you run the inversion again? Do you get the same parameter values and penalty? Why might they change? How could you find a more definitive answer? Could you implement a strategy to do that?

In [ ]:
# get on with implementing something




## 4. A simple Metropolis algorithm for MCMC

Markov Chain Monte Carlo (MCMC) approaches are effectively a random walk though parameter space, guided by probability. The most probable models are those that fit the data well. Typically, we try to maximize the log-probability of the model, which sounds difficult until you realize it is closely related to the total squared misfit (which took me years to realize, but never mind...)

Specifically, here we use a Metropolis algorithm to conduct this sampling of parameter space. There are a couple of rules that we follow here:

- Every iteration starts with a known set of parameters and their corresponding log-probability
- We then propose a random perturbation to those parameters and compute the log-probability for those
- If the probability of the proposed model is higher, we accept the new parameters and start a new iteration
- If the probability of the proposed model is lower, we draw a (log) random number and compare it to the (log) ratio of the probabilities of the two models; if the ratio is the larger of the two, then we accept the new parametes and start a new iteration
- If the new parameters do not pass these tests, we retain the original parameters and start a new iteration

Over multiple iterations, this will move the model parameters towards the part of parameter space that contains the 'good' models (ones that fit the data). At least, I hope so...

In [ ]:
# we will apply a very simple metropolis algorithm
# this is based on Iain Murray's 'dumb metropolis'
# - see here: https://homepages.inf.ed.ac.uk/imurray2/teaching/09mlss/
# - and here: https://homepages.inf.ed.ac.uk/imurray2/teaching/09mlss/slides.pdf

# define what we shall use as our inputs 
x=x_gps
data=v_gps
ndata=len(data)

# choose a random initial model state
init=pars_lb+(pars_ub-pars_lb)*np.random.rand(npars)

# some parameters for your metropolis run
niters=10000    # number of iterations
stepsigma=1  # a scaling for the size of possible parameter changes
               # (can be a scalar or an array of length npars)

# somewhere to store the output
samples=np.empty((niters,npars))    # all your accepted model states 
logprobs=np.empty((niters))         # all your log penalties
arate=0                            # acceptance counter

# ok, lets's get going
state = init                           # current model state
lp_state = logprobability(state, x, data)  # current log penalty

# loop through all your iterations
for ss in np.arange(niters):
    
    # propose a new model state and calculate its log probability
    prop = state + stepsigma*np.random.randn(npars);
    lp_prop = logprobability(prop, x, data);
    
    # test for acceptance
    if np.log(np.random.rand(1)) < (lp_prop-lp_state):
        # if you accept the change, update your counter, model state and log probability
        arate+=1
        state = prop
        lp_state = lp_prop
    
    # and record the result of the iteration for posterity
    samples[ss,:] = state
    logprobs[ss]= lp_state

arate = arate/niters

Note that we store all of the 'accepted' model parameters in an array, which means we can analyze them as an ensemble to see how they did. We can start by simply taking the mean, but we really ought to plot them and look at some histograms.

In [ ]:
print('mean s: ',np.average(samples[:,0]))
print('mean D: ',np.average(samples[:,1]))
print('mean v_shift: ',np.average(samples[:,2]))
print('fraction of accepted models: ',arate)

Consider plotting:

- the penalty function as it evolves over all of the iterations
- the parameter values as they evolve over all of the iterations
- histograms of the parameter values
- maybe a scatter plot between s and D

Do you see any differences between early models and late ones? Which ones might be more representative?

In [ ]:
# this space for thinking about some plots...




Note also that the algorithm takes as an input a value, called here `stepsigma` which is a scaling of the step size. What happens if you change it, and re-run the inversion? What else might you want to change?

In [ ]:
# maybe copy the MCMC code here and re-run with different input values, and re-plot the results?


